In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import yfinance as yf
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import msgpack
import warnings
from dataclasses import asdict, is_dataclass, dataclass
import os
import msgpack
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype
import scipy
import numpy as np

from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.databaseService.OutsourceLoader import OutsourceLoader

from src.simulation.SimulatePortfolio import SimulatePortfolio
from src.strategy.StratBuyAndHold import StratBuyAndHold
from src.simulation.ResultAnalyzer import ResultAnalyzer
from src.common.AssetFileInOut import AssetFileInOut
from src.mathTools.CurveAnalysis import CurveAnalysis


In [3]:
from src.common.YamlTickerInOut import YamlTickerInOut

snp500tickers = YamlTickerInOut("../src/tickerSelection").loadFromFile("snp500.yaml")["snp500tickers"]
alltickers = YamlTickerInOut("../src/stockGroups").loadFromFile("group_all.yaml")

In [4]:
assetG = AssetFileInOut("../src/database").loadFromFile('IRM')
ads=assetG.shareprice.iloc[-20:-1]["Close"]

print(ads)

ads.resample('B').mean().dropna()

print(ads)


2024-09-06 00:00:00-04:00    108.811302
2024-09-09 00:00:00-04:00    109.715736
2024-09-10 00:00:00-04:00    113.462715
2024-09-11 00:00:00-04:00    113.681366
2024-09-12 00:00:00-04:00    115.649277
2024-09-13 00:00:00-04:00    116.344994
2024-09-16 00:00:00-04:00    118.190002
2024-09-17 00:00:00-04:00    115.919998
2024-09-18 00:00:00-04:00    116.449997
2024-09-19 00:00:00-04:00    114.129997
2024-09-20 00:00:00-04:00    115.360001
2024-09-23 00:00:00-04:00    118.440002
2024-09-24 00:00:00-04:00    119.169998
2024-09-25 00:00:00-04:00    119.320000
2024-09-26 00:00:00-04:00    117.309998
2024-09-27 00:00:00-04:00    116.730003
2024-09-30 00:00:00-04:00    118.830002
2024-10-01 00:00:00-04:00    118.239998
2024-10-02 00:00:00-04:00    119.400002
Name: Close, dtype: float64
2024-09-06 00:00:00-04:00    108.811302
2024-09-09 00:00:00-04:00    109.715736
2024-09-10 00:00:00-04:00    113.462715
2024-09-11 00:00:00-04:00    113.681366
2024-09-12 00:00:00-04:00    115.649277
2024-09-13 0

In [20]:
assetG = AssetFileInOut("../src/database").loadFromFile('IRM')

a=assetG.shareprice.iloc[-100:-51]["Close"]
res=CurveAnalysis.thirdDegreeFit(a, "IRM")
x = np.arange(len(a))
y = a

# Perform cubic polynomial regression with covariance
coeff, covariance = np.polyfit(x, y, 3, cov=True)

#print(coeff)
#plt.plot(x,y)
#plt.plot(x,res)

coefficients = coeff
r_squared = res['R_Squared']
variance = res['Variance']

y.values.tolist()


[77.33584594726562,
 79.44706726074219,
 81.23272705078125,
 80.89730072021484,
 80.69012451171875,
 80.99595642089844,
 81.7358627319336,
 79.87129211425781,
 78.21388244628906,
 79.15110778808594,
 78.43091583251953,
 77.32598114013672,
 78.56903839111328,
 79.60491943359375,
 79.6443862915039,
 80.05873107910156,
 81.23272705078125,
 83.69910430908203,
 84.32064056396484,
 85.92871856689453,
 86.15563201904297,
 86.12603759765625,
 86.58970642089844,
 86.83634185791016,
 87.73084259033203,
 88.15821838378906,
 88.28742980957031,
 88.31724548339844,
 88.54583740234375,
 87.99919891357422,
 87.90974426269531,
 88.66510772705078,
 89.07260131835938,
 89.14217376708984,
 90.29508972167969,
 91.21940612792969,
 91.0405044555664,
 92.3027572631836,
 92.5611572265625,
 94.5688247680664,
 93.94267272949219,
 95.30430603027344,
 97.23246002197266,
 99.36932373046875,
 98.61396789550781,
 97.24239349365234,
 97.67970275878906,
 99.51840209960938,
 100.3234634399414]

In [39]:
from xgboost import XGBClassifier
# read data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2)
# create model instance
bst = MultiOutputClassifier(XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:softprob', num_class=2))
# fit model
y_train_2 = np.array([[a, a] for a in y_train])
bst.fit(X_train, y_train_2)
# make predictions
preds = bst.predict(X_test)


print(y_train_2-preds)

ValueError: operands could not be broadcast together with shapes (120,2) (30,2) 

In [ ]:
import pandas as pd

data = {
    'Date': ['1996-02-01 00:00:00-05:00', '1996-02-02 00:00:00-05:00', '1996-02-05 00:00:00-05:00', 
             '1996-02-06 00:00:00-05:00', '1996-02-07 00:00:00-05:00'],
    'Open': [1.236482, 1.226971, 1.236482, 1.217460, 1.217460],
    'High': [1.236482, 1.236482, 1.236482, 1.236482, 1.236482],
    'Low': [1.217459, 1.217460, 1.217460, 1.217460, 1.217460],
    'Close': [1.226971, 1.236482, 1.217460, 1.217460, 1.217460],
    'AdjClose': [1.236971, 1.236482, 1.227460, 1.207460, 1.217460]
    }

# Convert the dictionary to a pandas dataframe
shareprice = pd.DataFrame(data)
shareprice['Date'] = pd.to_datetime(shareprice['Date'])  # Convert the 'Date' column to datetime
shareprice.index = shareprice['Date']

slice_df = shareprice.iloc[0:2]
#slice_df = slice_df.resample('B').mean().dropna()
# Modify the slice
slice_df['Low'] = [100, 200]

# Check original DataFrame
print(slice_df)
print(shareprice)

                                               Date      Open      High  Low  \
Date                                                                           
1996-02-01 00:00:00-05:00 1996-02-01 00:00:00-05:00  1.236482  1.236482  100   
1996-02-02 00:00:00-05:00 1996-02-02 00:00:00-05:00  1.226971  1.236482  200   

                              Close  Adj Close  
Date                                            
1996-02-01 00:00:00-05:00  1.226971   1.236971  
1996-02-02 00:00:00-05:00  1.236482   1.236482  
                                               Date      Open      High  \
Date                                                                      
1996-02-01 00:00:00-05:00 1996-02-01 00:00:00-05:00  1.236482  1.236482   
1996-02-02 00:00:00-05:00 1996-02-02 00:00:00-05:00  1.226971  1.236482   
1996-02-05 00:00:00-05:00 1996-02-05 00:00:00-05:00  1.236482  1.236482   
1996-02-06 00:00:00-05:00 1996-02-06 00:00:00-05:00  1.217460  1.236482   
1996-02-07 00:00:00-05:00 1996-02

C:\Users\KILightTouch\AppData\Local\Temp\ipykernel_17432\849920140.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slice_df['Low'] = [100, 200]
